In [6]:
## Quantum–Classical Hybrid Solver for 1D Viscous Burgers' Equation — *Monika edition*

##**Author:** Monika Sharma

In [7]:

# impport Standard scientific Python libraries
import numpy as np                     # numerical arrays and linear algebra
import scipy.linalg as la              # some linear algebra helpers (SVD, norms)
import scipy.sparse as sp              # sparse matrices for CN scheme
import scipy.sparse.linalg as spla     # sparse linear solvers / factorization
import matplotlib.pyplot as plt        #  for plotting
from scipy.interpolate import interp1d # interpolation helper for comparison
import time                            # timing operations
import warnings                        # suppress minor warnings in notebook
warnings.filterwarnings('ignore')





In [19]:
# Qiskit core imports
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import Statevector

# Qiskit Aer simulator & noise models
from qiskit_aer import Aer, AerSimulator
from qiskit.providers.aer.noise import NoiseModel, depolarizing_error, pauli_error

# Measurement error mitigation (Ignis)
from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter

# Optimizers (classical)
from qiskit.algorithms.optimizers import COBYLA

# NOTE: Modern way to run instead of `execute`
def run_circuit(circuit, backend, shots=1024):
    compiled = transpile(circuit, backend)
    job = backend.run(compiled, shots=shots)
    return job.result()

# Example backend
backend = AerSimulator()
# Qiskit core imports
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import Statevector

# Qiskit Aer simulator & noise models
from qiskit_aer import Aer, AerSimulator
from qiskit.providers.aer.noise import NoiseModel, depolarizing_error, pauli_error

# Measurement error mitigation (Ignis)
from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter

# Optimizers (classical)
from qiskit.algorithms.optimizers import COBYLA

# NOTE: Modern way to run instead of `execute`
def run_circuit(circuit, backend, shots=1024):
    compiled = transpile(circuit, backend)
    job = backend.run(compiled, shots=shots)
    return job.result()

# Example backend
backend = AerSimulator()


ModuleNotFoundError: No module named 'qiskit.providers.aer'

In [16]:
#Display settings to make arrays readable
np.set_printoptions(precision=6, suppress=True)# Qiskit imports (quantum)
from qiskit import QuantumCircuit, Aer, execute, transpile
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import Statevector
from qiskit.providers.aer.noise import NoiseModel, depolarizing_error, pauli_error
from qiskit.utils import QuantumInstance
from qiskit.providers.aer import AerSimulator
from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter

# Optimizer from Qiskit (classical optimizer used to train circuit parameters)
from qiskit.algorithms.optimizers import COBYLA

# Display settings to make arrays readable
np.set_printoptions(precision=6, suppress=True)

ImportError: cannot import name 'Aer' from 'qiskit' (c:\Users\sharm\conda\envs\qunt\Lib\site-packages\qiskit\__init__.py)

In [ ]:
##grid helper: domain length L and number of grid points N (choose power of two for amplitude encoding)

def monika_make_grid(L=1.0, N=16):
    \"\"\"Create a periodic grid x and the grid spacing dx.\"\"\"
    x = np.linspace(0, L, N, endpoint=False)   # uniform points on [0,L)
    dx = x[1] - x[0]                            # grid spacing
    return x, dx

# two initial conditions: smooth sine wave and a Riemann step for shock formation
def monika_initial_sine(x):
    return np.sin(2*np.pi*x)  # smooth test function

def monika_initial_riemann(x, left=1.0, right=0.0, x0=0.5):
    # simple left-right step; mollified by small convolution to avoid discontinuity artifacts
    u = np.where(x < x0, left, right).astype(float)
    kernel = np.array([0.25, 0.5, 0.25])
    for _ in range(2):
        u = np.convolve(u, kernel, mode='same')
    return u

# normalization helper for amplitude encoding (amplitude vectors must be unit norm)
def monika_normalize_amplitudes(vec):
    vec = np.array(vec, dtype=complex)
    norm = np.linalg.norm(vec)
    if norm == 0:
        return vec
    return vec / norm

In [ ]:
## Classical reference solver (operator splitting)

We implement a split-step solver:
1. Advection (nonlinear): use SSP-RK3 on conservative form to handle shocks robustly.
2. Diffusion (linear): use Crank–Nicolson (implicit, stable) for the viscous term.
We will use the CN step as the *target* that the quantum variational circuit should approximate for the diffusion-only evolution.
""")))

cells.append(nbf.v4.new_code_cell(dedent("""
# === Classical solver functions with line-by-line comments ===

def monika_advective_flux(monika_u):
    # flux for Burgers equation: f(u) = 0.5 * u^2
    return 0.5 * monika_u**2

def monika_advect_rk3(monika_u, monika_dx, monika_dt):
    \"\"\"SSP-RK3 scheme applied to conservative form: du/dt = -d/dx (0.5 u^2)\"\"\"
    def rhs(u):
        f = monika_advective_flux(u)
        # centered difference for the derivative of flux (periodic)
        return - (np.roll(f, -1) - np.roll(f, 1)) / (2.0 * monika_dx)
    # SSP-RK3 stages
    u1 = monika_u + monika_dt * rhs(monika_u)
    u2 = 0.75*monika_u + 0.25*(u1 + monika_dt * rhs(u1))
    u3 = (1.0/3.0)*monika_u + (2.0/3.0)*(u2 + monika_dt * rhs(u2))
    return u3

def monika_build_cn_matrix(N, monika_dx, monika_nu, monika_dt):
    \"\"\"Build and factorize the Crank–Nicolson matrix once per timestep configuration.\"\"\"
    alpha = monika_nu * monika_dt / (2.0 * monika_dx * monika_dx)
    main = (1.0 + 2.0*alpha) * np.ones(N)
    off = -alpha * np.ones(N-1)
    # tridiagonal matrix with periodic boundary conditions
    A = sp.diags([off, main, off], offsets=[-1,0,1], shape=(N,N), format='csr').toarray()
    A[0, -1] = -alpha
    A[-1, 0] = -alpha
    A = sp.csr_matrix(A)
    # factorize for fast solves (lu(rhs) returns solution for RHS)
    lu = spla.factorized(A)
    return lu

def monika_diffuse_cn(monika_u, monika_dx, monika_nu, monika_dt, lu):
    \"\"\"Perform one Crank–Nicolson diffusion step given a prefactorized LU solver.\"\"\"
    alpha = monika_nu * monika_dt / (2.0 * monika_dx * monika_dx)
    rhs = (1.0 - 2.0*alpha) * monika_u + alpha * (np.roll(monika_u, -1) + np.roll(monika_u, 1))
    monika_u_np1 = lu(rhs)
    return monika_u_np1

def monika_classical_solver(u0, x, dx, nu, dt, t_final, return_every=10):
    \"\"\"Run the split-step solver to produce snapshots (used for reference and benchmarking).\"\"\"
    monika_u = u0.copy()
    monika_t = 0.0
    snapshots = []
    times = []
    # precompute LU factorization for the diffusion step
    lu = monika_build_cn_matrix(len(x), dx, nu, dt)
    step = 0
    while monika_t < t_final - 1e-12:
        # advection half-step (SSP-RK3)
        monika_u = monika_advect_rk3(monika_u, dx, dt/2.0)
        # diffusion full-step (Crank–Nicolson implicit)
        monika_u = monika_diffuse_cn(monika_u, dx, nu, dt, lu)
        # advection half-step (SSP-RK3)
        monika_u = monika_advect_rk3(monika_u, dx, dt/2.0)
        monika_t += dt
        step += 1
        if step % return_every == 0 or abs(monika_t - t_final) < 1e-12:
            snapshots.append(monika_u.copy())
            times.append(monika_t)
    return np.array(times), snapshots

In [ ]:
## Variational ansatz (monika_ansatz) — shallow circuit design

We define a shallow, low-depth ansatz using Qiskit's `TwoLocal` template. The ansatz has parameterized `ry` rotations and CX entanglers in a linear chain. This choice favors low depth while still allowing entanglement between neighboring qubits.
""")))

cells.append(nbf.v4.new_code_cell(dedent("""
def monika_make_ansatz(monika_qubits, reps=2):
    # TwoLocal creates repeated blocks of single-qubit rotations + entanglers
    monika_ansatz = TwoLocal(num_qubits=monika_qubits,
                             rotation_blocks='ry',
                             entanglement_blocks='cx',
                             reps=reps,
                             entanglement='linear',
                             insert_barriers=False)
    return monika_ansatz

# Example: show a small circuit for 4 qubits with 1 repetition (for demonstration)
# qc_demo = monika_make_ansatz(4, reps=1).decompose()
# display(qc_demo.draw(output='mpl'))

In [ ]:
## Loss function and training routine

We define a simple mean-square-error loss between target amplitude vector (classical CN diffusion result) and the output statevector amplitudes produced by the variational circuit applied to the initial amplitude-encoded state. For training we use the COBYLA optimizer (gradient-free) which is stable for low-parameter problems.
""")))

cells.append(nbf.v4.new_code_cell(dedent("""
# Use statevector simulator for training (noise-free) to get clean gradients of the objective.
monika_backend_sv = Aer.get_backend('statevector_simulator')

def monika_loss_for_params(monika_params, monika_base_statevec, monika_ansatz, monika_target_vector):
    \"\"\"Evaluate MSE loss between ansatz(monika_base_state) and monika_target_vector.\"\"\"
    # build circuit that initializes base state and applies the parameterized ansatz
    qc = QuantumCircuit(monika_ansatz.num_qubits)
    qc.initialize(monika_base_statevec, qc.qubits)
    # bind parameters to the ansatz template and append
    bound_ansatz = monika_ansatz.bind_parameters(monika_params)
    qc.compose(bound_ansatz, inplace=True)
    # execute on statevector simulator and extract final statevector
    result = execute(qc, monika_backend_sv).result()
    out_state = Statevector(result.get_statevector(qc))
    out_vec = np.array(out_state.data)
    # target amplitudes (normalized)
    tgt = monika_normalize_amplitudes(monika_target_vector)
    # return mean squared error (complex amplitudes -> absolute squared difference)
    mse = np.mean(np.abs(out_vec - tgt)**2)
    return mse

def monika_train_variational(monika_base_vector, monika_target_vector, monika_qubits, reps=2, maxiter=200):
    \"\"\"Train the monika_ansatz to map base_vector -> target_vector.\"\"\"
    monika_ansatz = monika_make_ansatz(monika_qubits, reps=reps)
    # Random initialization for parameters in [0, 2π)
    init_param = np.random.uniform(0, 2*np.pi, monika_ansatz.num_parameters)
    optimizer = COBYLA(maxiter=maxiter)
    # objective wrapper for optimizer
    def objective(x):
        return monika_loss_for_params(x, monika_base_vector, monika_ansatz, monika_target_vector)
    start = time.time()
    # optimizer returns (opt_params, final_value, nfev)
    res = optimizer.optimize(num_vars=monika_ansatz.num_parameters, objective=objective, initial_point=init_param)
    end = time.time()
    params_opt = res[0]
    final_loss = res[1]
    print(f\"Training finished in {end-start:.1f}s, final loss = {final_loss:.3e}\")
    return monika_ansatz, params_opt, final_loss
    

In [ ]:
## Noise model (Qiskit Aer) and simple measurement error mitigation

We build a simple noise model (depolarizing on 1- and 2-qubit gates) for simulation. We also prepare a measurement calibration (CompleteMeasFitter) that can be applied to mitigate readout errors in measurement-based experiments. This demonstrates basic error mitigation techniques you can run on real hardware.
""")))

cells.append(nbf.v4.new_code_cell(dedent("""
# Build a basic depolarizing noise model for demonstration
def monika_build_noise_model(p1=0.001, p2=0.01):
    noise_model = NoiseModel()
    # depolarizing errors for single- and two-qubit gates
    err1 = depolarizing_error(p1, 1)
    err2 = depolarizing_error(p2, 2)
    # attach to common gate labels (approximate)
    noise_model.add_all_qubit_quantum_error(err1, ['ry', 'rz', 'rx', 'h'])
    noise_model.add_all_qubit_quantum_error(err2, ['cx', 'cz'])
    return noise_model

# Simple measurement calibration circuits for N qubits (for measurement mitigation)
def monika_measurement_calibration(monika_qubits, backend):
    # Generate calibration circuits (2^n circuits) — costly for large n; small n is fine.
    meas_circuits, state_labels = complete_meas_cal(qubit_list=list(range(monika_qubits)), circlabel='mcal')
    # Execute calibration circuits on the chosen backend (noisy or real)
    job = execute(meas_circuits, backend=backend, shots=1024)
    cal_results = job.result()
    meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='mcal')
    return meas_fitter


In [ ]:
## Demo: train monika_ansatz to approximate one diffusion step (small N)

This cell performs the following:
- Defines a small grid (N=16), initial Riemann condition `monika_velocity0`
- Computes the exact Crank–Nicolson diffusion result `monika_u_diff` (target for training)
- Trains `monika_ansatz` (COBYLA) to map amplitude-encoded `monika_velocity0` -> `monika_u_diff`
""")))

cells.append(nbf.v4.new_code_cell(dedent("""
# === Demo parameters (small so training is fast) ===
monika_L = 1.0
monika_N = 16                   # must be a power of two for amplitude encoding
monika_qubits = int(np.log2(monika_N))
monika_nu = 1e-3                # viscosity
monika_tfinal = 0.02

# Create grid and initial condition (Riemann problem -> steep gradient)
monika_x, monika_dx = monika_make_grid(L=monika_L, N=monika_N)
monika_velocity0 = monika_initial_riemann(monika_x, left=1.0, right=0.0, x0=0.5)

# Choose a timestep dt (CFL-style heuristic)
monika_dt = 0.25 * min(monika_dx / 1.0, monika_dx*monika_dx / monika_nu)
print(f\"monika_N={monika_N}, monika_qubits={monika_qubits}, monika_dx={monika_dx:.6f}, monika_dt={monika_dt:.6e}\")

# Compute diffusion-only target using Crank–Nicolson (this is the 'truth' for diffusion)
monika_lu = monika_build_cn_matrix(monika_N, monika_dx, monika_nu, monika_dt)
monika_u_diff = monika_diffuse_cn(monika_velocity0, monika_dx, monika_nu, monika_dt, monika_lu)

# Build amplitude-encoded Statevectors for base and target
monika_base_state = monika_vec_to_statevector(monika_velocity0)
monika_target_state = monika_vec_to_statevector(monika_u_diff)

# Train variational ansatz to approximate diffusion step
print('Training monika_ansatz to approximate diffusion (may take ~1-2 minutes depending on your CPU)...')
monika_ansatz, monika_params_opt, monika_final_loss = monika_train_variational(monika_base_state, monika_u_diff, monika_qubits, reps=2, maxiter=200)

In [ ]:

bound_monika_ans = monika_ansatz.bind_parameters(monika_params_opt)
qc_monika_final = QuantumCircuit(monika_qubits)
qc_monika_final.initialize(monika_base_state, qc_monika_final.qubits)
qc_monika_final.compose(bound_monika_ans, inplace=True)

# Run on statevector simulator to get ideal output amplitudes
res = execute(qc_monika_final, Aer.get_backend('statevector_simulator')).result()
out_sv = Statevector(res.get_statevector(qc_monika_final))
out_vec = np.array(out_sv.data)

# Map amplitude vector back to classical velocity via simple energy scaling:
# choose scale s so that || s * out_vec.real ||_2 = || monika_u_diff ||_2
scale = np.linalg.norm(monika_u_diff) / (np.linalg.norm(out_vec.real) + 1e-16)
monika_u_quantum_hat = out_vec.real * scale

# Compute relative L2 error vs CN diffusion target
rel_err_quantum_vs_cn = la.norm(monika_u_quantum_hat - monika_u_diff) / (la.norm(monika_u_diff) + 1e-16)
print(f\"Relative L2 error (ideal quantum approximation vs CN target): {rel_err_quantum_vs_cn:.4f}\")

# Plot comparison: initial, classical CN diffusion target, and quantum approx
plt.figure(figsize=(8,4))
plt.plot(monika_x, monika_velocity0, '--', label='monika_velocity0 (initial)')
plt.plot(monika_x, monika_u_diff, label='monika_u_diff (classical CN target)')
plt.plot(monika_x, monika_u_quantum_hat, 'o-', label='monika_u_quantum_hat (variational output)', markersize=4)
plt.legend(); plt.xlabel('x'); plt.ylabel('u'); plt.title('Diffusion step: classical vs quantum approx'); plt.grid(True)
plt.show()


In [ ]:
## Noisy simulation and measurement error mitigation demo

We run the trained circuit in a noisy environment (Aer noise model) and then demonstrate a basic measurement calibration to correct readout errors. This is a simplified demonstration — on real hardware you'll want to use dedicated calibration jobs and advanced mitigation techniques.
""")))

cells.append(nbf.v4.new_code_cell(dedent("""
# Build a simple depolarizing noise model and simulate the final circuit under noise
monika_noise = monika_build_noise_model(p1=0.002, p2=0.01)
sim_backend = AerSimulator(method='density_matrix')

# Transpile circuit for backend (important for realistic gate set mapping)
qc_noisy = transpile(qc_monika_final, sim_backend)

# Run with noise model (density matrix simulator supports noise models)
job = sim_backend.run(qc_noisy, noise_model=monika_noise, shots=0)  # shots=0 for state emulation via density matrix
res_noisy = job.result()
# extract density matrix from result data (backend returns 'density_matrix' for simulator method)
try:
    rho = res_noisy.data(qc_noisy)['density_matrix']
    # take principal eigenvector of density matrix as approximate noisy pure state
    eigvals, eigvecs = la.eigh(rho)
    noisy_sv = eigvecs[:, np.argmax(eigvals)]
    out_vec_noisy = noisy_sv
    scale_noisy = np.linalg.norm(monika_u_diff) / (np.linalg.norm(out_vec_noisy.real) + 1e-16)
    monika_u_noisy_hat = out_vec_noisy.real * scale_noisy
    print('Simulated noisy run completed — computed scaled classical estimate from noisy state.')
except Exception as e:
    print('Could not extract density matrix from noisy result:', e)
    monika_u_noisy_hat = None

# Measurement calibration (small qubit count)
print('Preparing measurement calibration (this will run 2^n circuits; OK for n<=6).')
if monika_qubits <= 6:
    # Use the same simulator backend (with noise) to create calibration circuits and fitter
    meas_circuits, state_labels = complete_meas_cal(qubit_list=list(range(monika_qubits)), circlabel='mcal')
    cal_job = sim_backend.run(transpile(meas_circuits, sim_backend), noise_model=monika_noise, shots=2048)
    cal_results = cal_job.result()
    meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='mcal')
    print('Measurement calibration fitted. You can now apply meas_fitter.filter.apply to counts.')
else:
    meas_fitter = None
    print('Skipping measurement calibration (qubit count too large for exhaustive calibration).')

# Plot noisy vs ideal
plt.figure(figsize=(8,4))
plt.plot(monika_x, monika_u_diff, label='CN target')
plt.plot(monika_x, monika_u_quantum_hat, label='ideal quantum approx')
if monika_u_noisy_hat is not None:
    plt.plot(monika_x, monika_u_noisy_hat, 'x-', label='noisy quantum approx')
plt.legend(); plt.xlabel('x'); plt.ylabel('u'); plt.title('Noisy vs ideal quantum approx'); plt.grid(True)
plt.show()

In [ ]:
## Full hybrid time-stepping: classical advection + trained variational quantum diffusion

We will run a short time integration where:
- advection is handled classically (SSP-RK3)
- diffusion step is approximated by applying the trained `monika_ansatz` unitary to the amplitude-encoded state, then decoding back to a classical velocity via scaling
This demonstrates the full hybrid workflow and compares the final state to a fully classical solver.
""")))

cells.append(nbf.v4.new_code_cell(dedent("""
# Hybrid driver: apply trained variational diffusion (ideal) to a classical vector
def monika_apply_variational_diffusion(monika_u_vec, monika_ansatz, monika_params, monika_scale_target=None):
    # amplitude-encode the classical vector into statevector
    monika_sv = monika_normalize_amplitudes(monika_u_vec)
    # build circuit: initialize -> apply ansatz with parameters
    qc = QuantumCircuit(monika_qubits)
    qc.initialize(monika_sv, qc.qubits)
    qc.compose(monika_ansatz.bind_parameters(monika_params), inplace=True)
    # simulate statevector output
    res = execute(qc, Aer.get_backend('statevector_simulator')).result()
    out_sv = Statevector(res.get_statevector(qc))
    out_vec = np.array(out_sv.data)
    # scale back to classical velocity amplitude using provided target norm (if given) or input norm
    if monika_scale_target is None:
        scale = np.linalg.norm(monika_u_vec) / (np.linalg.norm(out_vec.real) + 1e-16)
    else:
        scale = monika_scale_target
    return out_vec.real * scale

# Run short hybrid integration
monika_t = 0.0
monika_u = monika_velocity0.copy()
monika_steps = int(np.ceil(monika_tfinal / monika_dt))
monika_snapshots = [monika_u.copy()]
monika_times = [0.0]

for s in range(monika_steps):
    # advection half-step (classical)
    monika_u = monika_advect_rk3(monika_u, monika_dx, monika_dt/2.0)
    # quantum diffusion approx
    monika_u = monika_apply_variational_diffusion(monika_u, monika_ansatz, monika_params_opt, monika_scale_target=None)
    # advection half-step (classical)
    monika_u = monika_advect_rk3(monika_u, monika_dx, monika_dt/2.0)
    monika_t += monika_dt
    monika_snapshots.append(monika_u.copy())
    monika_times.append(monika_t)

# For reference, run full classical solver on same grid
monika_times_ref, monika_snaps_ref = monika_classical_solver(monika_velocity0, monika_x, monika_dx, monika_nu, monika_dt, monika_tfinal, return_every=10)
monika_u_ref_final = monika_snaps_ref[-1]
monika_u_hybrid_final = monika_snapshots[-1]

# Compare final profiles
rel_err_hybrid = la.norm(monika_u_hybrid_final - monika_u_ref_final) / (la.norm(monika_u_ref_final) + 1e-16)
print(f\"Hybrid final relative L2 error vs classical ref: {rel_err_hybrid:.4f}\")

plt.figure(figsize=(8,4))
plt.plot(monika_x, monika_u_ref_final, label='classical reference')
plt.plot(monika_x, monika_u_hybrid_final, 'o-', label='hybrid (variational diffusion)', markersize=4)
plt.legend(); plt.xlabel('x'); plt.ylabel('u'); plt.title('Hybrid solver final comparison'); plt.grid(True)
plt.show()